In [90]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## General Stuff
################################################################################
from timeUtils import clock, elapsed
from listUtils import getFlatList
from time import sleep
from pandas import DataFrame, Series
from ioUtils import getFile, saveFile
from searchUtils import findDirs, findNearest, findAll
from fileUtils import getDirBasics, getBaseFilename
from unicodedata import normalize
from fsUtils import moveDir, setDir, mkDir, isDir, removeDir
from pandasUtils import getRowDataByColValue, getRowData
import operator
from glob import glob
from os.path import join
from collections import Counter


################################################################################
## Mp3 Stuff
################################################################################
from mp3id import mp3ID


################################################################################
## Database Stuff
################################################################################
from discogsBase import discogs
from mainDB import getAlbumTypes
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import getArtistAlbumsDB, discConv


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap
from musicBase import myMusicBase
from musicMatch import myMatchedMusic


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-24 19:48:48.376806


# My Music Database Map

In [45]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()

Current Time is Fri Apr 24, 2020 18:39:41 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Fri Apr 24, 2020 18:40:13 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 31 seconds.
Summary Statistics For DB: Discogs
    Found 718816 ID -> Name entries
    Found 660527 Name -> ID entries
    Found 0 Albums
Current Time is Fri Apr 24, 2020 18:40:14 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Fri Apr 24, 2020 18:40:15 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
==================================

# Music Base

In [91]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase()
mmb.findArtistAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']
Current Time is Fri Apr 24, 2020 19:48:53 for Finding All Artist Albums
Current Time is Fri Apr 24, 2020 19:49:32 for Done with Finding All Artist Albums
Process [Done with Finding All Artist Albums] took 39 seconds.


# Match My Artists

In [92]:
from timeUtils import clock, elapsed
from musicBase import myMusicBase
from ioUtils import saveFile

class myMatchedMusic:
    def __init__(self, mdb, debug=False):
        self.debug = debug
        self.mdb   = mdb
        self.mmb   = myMusicBase()
        
        self.unknownArtists = {}
        self.artistAlbums   = {}
        
        self.matchedAlbums  = {}
        
        
    def setMusicBase(self, mmb):
        self.mmb = mmb
        
        
    def getAlbumStatus(self, force=False):
        self.artistAlbums = self.mmb.getArtistAlbums(force=force)
        

    def getArtistStatus(self):
        start, cmt = clock("Matching All Music Artists")

        ######################################################################
        #### Loop Over My Artists and Paths
        ######################################################################
        for primeDir in self.mmb.getPrimeDirectories():
            for artistName, artistPrimeDirs in self.mmb.getArtistPrimeDirMap(primeDir).items():
                if self.debug:
                    print("{0: <50}{1}".format(artistName,artistPrimeDirs))


                ######################################################################
                #### Get Database IDs
                ######################################################################
                isKnown = self.mdb.isKnown(artistName)
                if isKnown is False:
                    self.unknownArtists[artistName] = artistPrimeDirs
                    if self.debug:
                        print("\tUnknown (All)     --> {0}".format(artistName))
                        

        elapsed(start, cmt)
        print("Found {0} unknown artists".format(len(self.unknownArtists)))
        print("Found {0} total artists".format(len(self.artistAlbums)))

        
        
    def getUnknownArtists(self):
        return self.unknownArtists
    
    
    
    def getArtistNameMatchedDirs(self):
        self.artistMatchedDirs = {}
        for primeDir in self.mmb.getPrimeDirectories():
            self.artistMatchedDirs.update(self.mmb.getArtistPrimeDirMap(primeDir))
            

    def formatAlbum(self, albumName, albumType):
        if albumType == 3:
            retval = albumName.replace("(Single)", "")
            retval = retval.replace("(EP)", "")
            retval = retval.strip()
            return retval
        return albumName
    
    
    
    def matchMyMusicAlbumsByArtist(self, db, artistName, albumType=1, ratioCut=0.95, maxCut=0.1):

        matchedAlbums = {}
        

        ######################################################################
        #### Get Artist Album Data
        ######################################################################
        artistAlbumsData = self.mmb.getArtistAlbumsByArtist(artistName)
    
        if artistAlbumsData.getNumUnmatched() == 0:
            return matchedAlbums
            
            
        ######################################################################
        #### Loop Over Artist Name <-> Prime Map Items
        ######################################################################
        if self.mdb.isKnown(artistName) is True:
            myMusicData = self.mdb.getArtistData(artistName)
            try:
                artistID = myMusicData[db]["ID"]
            except:
                return matchedAlbums
        else:
            return matchedAlbums
            


        ######################################################################
        #### Get Database Albums
        ######################################################################
        artistDBAlbumsFromID = self.mdb.getArtistAlbumsFromID(db, artistID)


        ######################################################################
        #### Loop over my albums
        ######################################################################
        for dirval, unMatchedAlbums in artistAlbumsData.getUnmatched().items():
            for myAlbumName in unMatchedAlbums:

                bestMatchVal = {"Ratio": ratioCut, "Dir": None, "Album": None}
                for mediaType, mediaTypeAlbums in artistDBAlbumsFromID.items():
                    if self.debug:
                        print("\tMy album: {0}".format(myAlbumName))
                    myFormattedAlbum = self.formatAlbum(myAlbumName, mediaType)

                    ma = matchAlbums(cutoff=ratioCut)
                    ma.match([myFormattedAlbum], mediaTypeAlbums)

                    if ma.maxval < ratioCut or ma.maxval > ratioCut+maxCut:
                        continue
                    if ma.maxval < bestMatchVal["Ratio"]:
                        continue

                    bestMatch = ma.getBestMatch(myFormattedAlbum)

                    bestMatchVal = {"Ratio": ma.maxval, "Dir": dirval, "Album": {"Name": bestMatch["Name"], "Code": bestMatch["Code"], "MediaType": mediaType}}
                    matchedAlbums[myAlbumName] = bestMatchVal
                    #print("{0: <30}{1: <15}{2: <30} --> {3}".format(artistName, db, myAlbumName, bestMatchVal["Album"]))
                    #bestMatchVal["Match"].show(debug=True)
                    
        return matchedAlbums

                
    
    def matchMyMusicAlbums(self, db, albumType=1, ratioCut=0.95, maxCut=0.1):
        self.matchedAlbums = {}

        start, cmt = clock("Checking for Albums Matches Against {0} DB".format(db))
        
        
        print("{0: <40}{1: <15}{2: <45} --> {3}".format("Artist", "Database", "Album Name", "Matched Album"))

        ######################################################################
        #### Get Map of Artists and Unmatched Albums
        ######################################################################
        artistNames = self.mmb.getArtists()
        #artistAlbums = self.mmb.getArtistAlbums()


        ######################################################################
        #### Loop Over Artist Name <-> Prime Map Items
        ######################################################################
        for artistName in artistNames:
            matchedAlbums = self.matchMyMusicAlbumsByArtist(db, artistName, albumType, ratioCut, maxCut)
            if len(matchedAlbums) > 0:
                if self.matchedAlbums.get(db) is None:
                    self.matchedAlbums[db] = {}
                self.matchedAlbums[db][artistName] = matchedAlbums
                for myAlbumName,bestMatchVal in matchedAlbums.items():
                    print("{0: <40}{1: <15}{2: <45} --> {3}".format(artistName, db, myAlbumName, bestMatchVal["Album"]))

            
        elapsed(start, cmt)

        saveFile(ifile="myMusicAlbumMatch.yaml", idata=self.matchedAlbums, debug=True)
        print("Found {0} music <-> discogs albums maps".format(len(self.matchedAlbums)))
        


    def getMatchedDirName(self, albumName, albumID, dbKey):
        if dbKey == "AllMusic":
            matchedDirName = " :: ".join([discConv(albumName), "[AM-{0}]".format(albumID)])
        elif dbKey == "MusicBrainz":
            matchedDirName = " :: ".join([discConv(albumName), "[MB-{0}]".format(albumID)])
        elif dbKey == "Discogs":
            matchedDirName = " :: ".join([discConv(albumName), "[DC-{0}]".format(albumID)])
        elif dbKey == "AceBootlegs":
            matchedDirName = " :: ".join([discConv(albumName), "[AB-{0}]".format(albumID)])
        elif dbKey == "RateYourMusic":
            matchedDirName = " :: ".join([discConv(albumName), "[RM-{0}]".format(albumID)])
        elif dbKey == "LastFM":
            matchedDirName = " :: ".join([discConv(albumName), "[LM-{0}]".format(albumID)])
        elif dbKey == "DatPiff":
            matchedDirName = " :: ".join([discConv(albumName), "[DP-{0}]".format(albumID)])
        elif dbKey == "RockCorner":
            matchedDirName = " :: ".join([discConv(albumName), "[RC-{0}]".format(albumID)])
        elif dbKey == "CDandLP":
            matchedDirName = " :: ".join([discConv(albumName), "[CL-{0}]".format(albumID)])
        elif dbKey == "MusicStack":
            matchedDirName = " :: ".join([discConv(albumName), "[MS-{0}]".format(albumID)])
        else:
            raise ValueError("dbKey {0} not recognized!".format(dbKey))
        
        return matchedDirName        


    def getUnMatchedDirName(self, matchedDirName, mediaDirType):
        vals = matchedDirName.split(" :: ")
        if len(vals) == 2:
            albumName  = vals[0]
            albumIDval = vals[1]
            try:
                albumID = int(albumIDval[(albumIDval.find("[")+3):albumIDval.rfind("]")])
            except:
                raise ValueError("Could not extract album ID from {0}".format(albumIDval))

            if sum([x in mediaDirType for x in ["Single", "EP"]]) > 0:
                albumName = "{0} (Single)".format(albumName)

            if sum([x in mediaDirType for x in ["Mix", "MixTape"]]) > 0:
                albumName = "{0} (MixTape)".format(albumName)

            return albumName
        else:
            raise ValueError("Could not extract album name from {0}".format(matchedDirName))
            
    
        
    def moveMyMatchedMusicAlbums(self, show=False):
        rename = True
        albumsToMove = getFile(ifile="myMusicAlbumMatch.yaml")
        print("Found {0} music <-> discogs albums maps".format(len(albumsToMove)))
        
        for db, dbValues in albumsToMove.items():
            if dbValues is None:
                continue
            for artistName, artistAlbums in dbValues.items():
                print("==>",artistName)
                for myAlbumName,albumVals in artistAlbums.items():
                    dirval   = albumVals["Dir"]
                    albumVal = albumVals["Album"]
                    ratio    = albumVals["Ratio"]
                    
                    dbAlbumName = albumVal["Name"]
                    dbAlbumCode = albumVal["Code"]
                    mediaType   = albumVal["MediaType"]


                    matchedDir = setDir(dirval, "Match")
                    mkDir(matchedDir)
                    
                    srcName = myAlbumName
                    srcDir  = setDir(dirval, srcName)
                    if not isDir(srcDir):
                        print("{0} does not exist".format(srcDir))
                        continue

                    mediaDir = setDir(matchedDir, discConv(mediaType))
                    mkDir(mediaDir)

                    if rename is True:
                        dstName = self.getMatchedDirName(discConv(dbAlbumName), dbAlbumCode, db)
                    else:
                        dstName = self.getMatchedDirName(myAlbumName, dbAlbumCode, db)

                    if show is True:
                        print('\t{0}'.format(mediaDir))
                        print("\t\t[{0}]".format(srcName))
                        print("\t\t[{0}]".format(dstName))
                        continue


                    dstDir  = setDir(mediaDir, dstName)
                    if isDir(dstDir):
                        print("{0} already exists".format(dstDir))
                        continue

                    print("\tMoving {0}  --->  {1}".format(srcDir, dstDir))
                    moveDir(srcDir, dstDir, debug=True)

In [93]:
mmm = myMatchedMusic(mdb)
mmm.getArtistStatus()
mmm.setMusicBase(mmb)
#mmm.getUnknownArtists()

My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']
Current Time is Fri Apr 24, 2020 19:49:32 for Matching All Music Artists
Current Time is Fri Apr 24, 2020 19:49:32 for Done with Matching All Music Artists
Process [Done with Matching All Music Artists] took 187.0 millseconds.
Found 1 unknown artists
Found 0 total artists


In [94]:
##dR=0.1
##rC=0.1
##mmm.matchMyMusicAlbumsByArtist("AllMusic", "David Gray", albumType=1, ratioCut=rC, maxCut=rC+dR)

In [95]:
dR = 0.1
rC = 0.8

mmm.matchMyMusicAlbums(db="AllMusic", albumType=1, ratioCut=rC, maxCut=rC+dR)

Current Time is Fri Apr 24, 2020 19:49:32 for Checking for Albums Matches Against AllMusic DB
Artist                                  Database       Album Name                                    --> Matched Album
Artist ID [0000691561] is not found in Albums DB [AllMusic]
Choo Choo Train                         AllMusic       Briar High                                    --> {'Name': 'Briar High', 'Code': '0000960170', 'MediaType': 'Album'}
Church Of Betty                         AllMusic       Comedy Of Animals                             --> {'Name': 'Comedy of Animals', 'Code': '0000967534', 'MediaType': 'Album'}
David Parmley                           AllMusic       What We Leave Behind                          --> {'Name': 'What We Leave Behind', 'Code': '0000587955', 'MediaType': 'Album'}
Disembowelment                          AllMusic       Transcendence Into The Peripheral             --> {'Name': 'Transcendence into the Peripheral', 'Code': '0000176013', 'MediaType': 'Album'}

Unspun                                  AllMusic       Unspun                                        --> {'Name': 'Unspun', 'Code': '0000939689', 'MediaType': 'Album'}
Current Time is Fri Apr 24, 2020 19:49:53 for Done with Checking for Albums Matches Against AllMusic DB
Process [Done with Checking for Albums Matches Against AllMusic DB] took 20 seconds.
Saving data to myMusicAlbumMatch.yaml
Saved data to myMusicAlbumMatch.yaml
  --> This file is 11.1kB.
Found 1 music <-> discogs albums maps


In [96]:
mmm.moveMyMatchedMusicAlbums(show=True)

Found 1 music <-> discogs albums maps
==> 311


NameError: name 'albumID' is not defined

In [21]:
moveMatchedDirectories(myMusicAlbumMatch, rename=True, show=False)

NameError: name 'myMusicAlbumMatch' is not defined

In [ ]:

    mediaTypes = getAlbumTypes(dbKey, albumType)
    for mediaType in mediaTypes:
        if dbAlbumsData.get(mediaType) is None:
            continue
        mediaAlbums = dbAlbumsData[mediaType]
        for albumID, dbAlbumName in mediaAlbums.items():
            convAlbumName = discConv(dbAlbumName)
            s = SequenceMatcher(None, myAlbumName, convAlbumName)
            ratio = round(s.ratio(), 3)
            if debug:
                print("\tMy Album: {0: <30}\tMatch: {1: <30}\tRatio: {2}".format(myAlbumName, convAlbumName, ratio))
            if ratio > ratioCut and ratio <= maxCut:
                if maxRatio is None:
                    maxRatio = ratio
                if ratio < maxRatio:
                    continue
                maxRatio  = ratio
                albumData = [albumID, dbAlbumName, mediaType]
                

In [17]:
tmpA = {'A': ['y'], 'C': [1]}
tmpB = {'A': ['x'], 'B': []}

In [18]:
import collections

def dict_merge(dct, merge_dct):
    """ Recursive dict merge. Inspired by :meth:``dict.update()``, instead of
    updating only top-level keys, dict_merge recurses down into dicts nested
    to an arbitrary depth, updating keys. The ``merge_dct`` is merged into
    ``dct``.
    :param dct: dict onto which the merge is executed
    :param merge_dct: dct merged into dct
    :return: None
    """
    for k, v in merge_dct.items():
        if (k in dct and isinstance(dct[k], dict)
                and isinstance(merge_dct[k], collections.Mapping)):
            dict_merge(dct[k], merge_dct[k])
        else:
            dct[k] = merge_dct[k]

In [19]:
dict_merge(tmpA, tmpB)

In [21]:
tmpB

{'A': ['x'], 'B': []}

In [ ]:
mmb = myMusicBase()
mmb.getArtistAlbums()

In [ ]:

        artistNameMatchedDirs = self.getArtistNameMatchedDirs()
        for artistName, artistPrimeDirs in artistNameMatchedDirs.items():
            matchVal = matchArtistNameAlbum(artistName, artistPrimeDirs, mdb, db, ratioCut, maxCut, albumType, debug=False)
            if isinstance(matchVal, dict) and len(matchVal) > 0:
                if myMusicAlbumMatch.get(db) is None:
                    myMusicAlbumMatch[db] = {}
                if myMusicAlbumMatch[db].get(artistName) is None:
                    myMusicAlbumMatch[db][artistName] = {}
                myMusicAlbumMatch[db][artistName].update(matchVal)


        elapsed(start, cmt)

        from ioUtils import saveFile
        saveFile(ifile="myMusicAlbumMatch.yaml", idata=myMusicAlbumMatch, debug=True)
        print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumMatch)))
    

In [ ]:
mmm.getArtistNameMatchedDirs()

***
***

***
***

## Analyze Fully Unknown Music Artists

In [20]:
def forceRename(myMusicAlbumMatch, test=True):
    for dbKey, dbValues in myMusicAlbumMatch.items():
        if dbValues is None:
            continue
        print("Found {0} artist entries".format(len(dbValues)))
        for artistName, artistAlbums in dbValues.items():
            first = False
            for myAlbumName,albumVals in artistAlbums.items():
                dirval   = albumVals["Dir"]
                albumVal = albumVals["Album"]
                albumID, dbAlbumName, mediaType = albumVal

                dbAlbumNameConv = discConv(dbAlbumName)
                if myAlbumName != dbAlbumNameConv:
                    if myAlbumName.upper() == dbAlbumNameConv.upper():
                        continue
                    srcDir = setDir(dirval, myAlbumName)
                    dstDir = setDir(dirval, discConv(dbAlbumName))
                else:
                    continue

                if isDir(dstDir):
                    if first is False:
                        print("\n==>",artistName)
                        first = True
                    print("[{0}] and [{1}] already exists for artist {2}".format(srcDir, dstDir, artistName))

                if first is False:
                    print("\n==>",artistName)
                    first = True

                print("\tMoving {0}  --->  {1}".format(srcDir, dstDir))
                if test is False:
                    moveDir(srcDir, dstDir, debug=True)
                

def showArtist(artistName, discogsArtist):
    print("\t{0: <30} ---> {1}".format(artistName, discogsArtist))
    
def showAlbum(myAlbumName):
    print("\t\tMy Album: {0}".format(myAlbumName))
    
def showMatch(myAlbumName, albumData):
    print("\t\t   Match: {0}".format(albumData))

    
def matchMyAlbum(myAlbumName, dbAlbumsData, cuts, dbKey, albumType, debug=False):
    maxRatio  = None
    albumData = None
    ratioCut,maxCut = cuts
    if maxCut is None:
        maxCut = 1.0
        
    mediaTypes = getAlbumTypes(dbKey, albumType)
    for mediaType in mediaTypes:
        if dbAlbumsData.get(mediaType) is None:
            continue
        mediaAlbums = dbAlbumsData[mediaType]
        for albumID, dbAlbumName in mediaAlbums.items():
            convAlbumName = discConv(dbAlbumName)
            s = SequenceMatcher(None, myAlbumName, convAlbumName)
            ratio = round(s.ratio(), 3)
            if debug:
                print("\tMy Album: {0: <30}\tMatch: {1: <30}\tRatio: {2}".format(myAlbumName, convAlbumName, ratio))
            if ratio > ratioCut and ratio <= maxCut:
                if maxRatio is None:
                    maxRatio = ratio
                if ratio < maxRatio:
                    continue
                maxRatio  = ratio
                albumData = [albumID, dbAlbumName, mediaType]
                    
    if isinstance(maxRatio, float):
        retRatio = round(maxRatio,2)
    else:
        retRatio = None
        
    return albumData, retRatio


def showMatchedMoves(myMusicAlbumMatch, rename=False):
    for dbKey, dbValues in myMusicAlbumMatch.items():
        if dbValues is None:
            continue
        for artistName, artistAlbums in dbValues.items():
            print("==>",artistName)
            for myAlbumName,albumVals in artistAlbums.items():
                dirval   = albumVals["Dir"]
                albumVal = albumVals["Album"]
                albumID, dbAlbumName, mediaType = albumVal

                srcDir = setDir(dirval, myAlbumName)
                if rename is True:
                    dstName = getMatchedDirName(discConv(dbAlbumName), albumID, dbKey)
                else:
                    dstName = getMatchedDirName(myAlbumName, albumID, dbKey)   
    

    
def unMatchArtist(artistName):
    from os.path import join
    from fsUtils import removeDir, isFile, setFile, removeFile

    for musicDir in getMatchedDirs():
        dirval        = join(musicDir, getPrimeDirectory(artistName), artistName)
        if not isDir(dirval):
            continue
            
        matchedDir    = setDir(dirval, "Match")
        mediaTypeDirs = findDirs(matchedDir)
        for mediaTypeDir in mediaTypeDirs:
            for matchDir in findDirs(mediaTypeDir):
                albumName = getUnMatchedDirName(getDirBasics(matchDir)[-1], mediaTypeDir)

                srcDir = matchDir
                dstDir = setDir(dirval, albumName)
                if isDir(dstDir):
                    i = 0
                    while not isDir(dstDir):
                        dstDir = "{0} [Fix-{1}]".format(setDir(dirval, albumName), i)
                        i += 1

                moveDir(srcDir, dstDir, debug=True)

            if isDir(mediaTypeDir):
                DS_Store = setFile(mediaTypeDir, ".DS_Store")
                if isFile(DS_Store):
                    removeFile(DS_Store, debug=True)
                removeDir(mediaTypeDir, debug=True)

        if isDir(matchedDir):
            DS_Store = setFile(matchedDir, ".DS_Store")
            if isFile(DS_Store):
                removeFile(DS_Store, debug=True)
            removeDir(matchedDir, debug=True)
            
            
def analyzeFullyUnknownArtists(mdb, matchedResults, ratioCut=0.65, maxCut=None):
    maxCut = 1.0
    myMusicMatch = {}

    start, cmt = clock("Checking for Artist Matches")
            
    ma = matchAlbums()



    ######################################################################
    #### Loop Over Artist Name <-> Prime Map Items
    ######################################################################
    artistNameMatchedDirs = getArtistNameMatchedDirs()
    for artistName, artistPrimeDirs in artistNameMatchedDirs.items():
        if matchedResults["FullyUnknown"].get(artistName) is None:
            continue
        print(artistName)
        
        myMusicAlbums = []
        for dirval in artistPrimeDirs:
            myMusicAlbums += getMyMusicAlbums(dirval, returnNames=True)
        print(artistName,'\t',len(myMusicAlbums))
        artistAlbums        = mdb.getArtistAlbums(artistName, num=10, cutoff=0.7)
        
        for db,dbdata in artistAlbums.items():
            print("\t{0: <20}".format(db))
            for artist,IDs in dbdata.items():
                for artistID,artistIDData in IDs.items():
                    artistIDAlbums = getFlattenedArtistAlbums(artistIDData)                    
                    ma.match(artistIDAlbums, myMusicAlbums)
                    if ma.near >= 1 and ma.maxval >= 0.8:
                        if myMusicMatch.get(artistName) is None:
                            myMusicMatch[artistName] = {}
                            
                        if myMusicMatch[artistName].get(db) is None:
                            myMusicMatch[artistName][db] = {"ID": str(artistID), "Score": [ma.near, ma.score, ma.maxval]}
                        elif ma.near > myMusicMatch[artistName][db]["Score"][0] or ma.maxval > myMusicMatch[artistName][db]["Score"][2]:
                            myMusicMatch[artistName][db] = {"ID": str(artistID), "Score": [ma.near, ma.score, ma.maxval]}
                            
                        print("\t{0: <20}{1: <20}{2: <50}{3: <20}{4}\tMatch!!".format(db,artist,artistID,len(artistIDAlbums), [ma.near, ma.score, ma.maxval]))
                        

    from ioUtils import saveFile
    saveFile(ifile="myMusicArtistMatch.yaml", idata=myMusicMatch, debug=True)
    print("Found {0} music <-> discogs artist maps".format(len(myMusicMatch)))
    
    


In [ ]:
analyzeFullyUnknownArtists(mdb, matchedResults)

In [ ]:
myMusicMatch = getFile(ifile="myMusicArtistMatch.yaml")
print("Found {0} music <-> discogs artist maps".format(len(myMusicMatch)))

In [ ]:
if len(myMusicMatch) > 0:
    for artistName, artistValues in myMusicMatch.items():
        print(artistName)
        for db, dbEntry in artistValues.items():
            print('\t',db,'\t',dbEntry)
            mdb.add(artistName, db, dbEntry["ID"])
    mdb.save()

***
***

# Match Albums

In [ ]:
mmm = myMatchedMusic(mdb)
mmm.getStatus()

In [ ]:
dR=0.1
rC=0.9

mmm.matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="AllMusic", albumType="All")
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="MusicBrainz", albumType=3)
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="Discogs", albumType=3)
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="AceBootlegs", albumType=1) ## Max 1
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="LastFM", albumType=1) ## Max 1
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="RateYourMusic", albumType="All")
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="DatPiff", albumType=1) ## Max 1
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="RockCorner", albumType="All") ## Max 1
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="CDandLP", albumType="All") ## Max 1
#matchMyMusicAlbums(ratioCut=rC, maxCut=rC+dR, db="MusicStack", albumType="All") ## Max 1

***
***
# Move To Matched

In [ ]:
myMusicAlbumMatch = getFile(ifile="myMusicAlbumMatch.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumMatch)))

In [ ]:
moveMatchedDirectories(myMusicAlbumMatch, rename=True, show=True)

In [ ]:
moveMatchedDirectories(myMusicAlbumMatch, rename=True, show=False)

***
***
***

# UnMatch Artist

In [ ]:
unMatchArtist("The J. Geils Band")

In [ ]:
######################################################################
#### Loop Over Prime Directories
######################################################################
for primeDir in getPrimeDirectories():
    artistPrimeDirMap = getArtistPrimeDirMap(primeDir)

    ######################################################################
    #### Loop Over Artist Name <-> Prime Map Items
    ######################################################################
    for artistName, artistPrimeDirs in artistPrimeDirMap.items():
        unMatchArtist(artistName)
